In [1]:
from mlchartist.preprocessing import train_test_split

import pandas as pd
import numpy as np

In [2]:
apple = pd.read_csv('../../raw_data/processed/aapl.csv')
google = pd.read_csv('../../raw_data/processed/googl.csv')
amzn = pd.read_csv('../../raw_data/processed/amzn.csv')

joined_df = pd.DataFrame()
joined_df = joined_df.append(apple)
joined_df = joined_df.append(google)
joined_df = joined_df.append(amzn)

print('joined_df', len(joined_df))
print('')


apple_train, apple_test = train_test_split(apple, '3Y')
google_train, google_test = train_test_split(google, '3Y')
amazon_train, amazon_test = train_test_split(amzn, '3Y')

print('trains', len(apple_train) + len(google_train) + len(amazon_train))
print('tests', len(apple_test) + len(google_test) + len(amazon_test))

joined_df 19103

trains 16850
tests 2253


#1
pass scaler --> use a scaler
scaler=None

#2
get_full_ds_scaler + train/test
- full 100 NASDAQ
- holdout 10 companies --> holdount=[]

split function(split=False)
- input X companies
- X companies -> train/test sets
- remove outliers on train
- fit scaler on train
output - scaler, train, test



fit_train_scaler
- input train
- remove outliers
- fit scaler
- output scaler


90 compainies --> 15 companies X 6 models

build_arrays_new
- inputs -> full dataframe (without holdout), take in train/test/scaler

- split the data -> train/test
- if scaler not provided then call fit_train_scaler
- scale the train set


In [3]:
def train_test_split_multiple_companies(df, test_set_size):
    """
    Split the preprocessed stock data of multiple companies into a train and test dataset
    INPUT: the dataframe to be split, and size of the test set in months or years ('3M' or '2Y')
    OUTPUT: returns a train_set and test_set dataframe, index is set to the date

    EXAMPLE: train_set, test_set = train_test_split_multiple_companies(input_df, '3Y')  --> puts last 3 years in test_set
    """
    train_set = pd.DataFrame()
    test_set = pd.DataFrame()
    for ticker in df['ticker'].unique():
        company_df = df[df['ticker'] == ticker]
        temp_train_set, temp_test_set = train_test_split(company_df, test_set_size)
        train_set = train_set.append(temp_train_set)
        test_set = test_set.append(temp_test_set)
    return train_set, test_set

In [4]:
train_set, test_set = train_test_split_multiple_companies(joined_df, '3Y')

c:\users\owner\.venvs\lewagon\lib\site-packages\mlchartist\preprocessing.py:125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = pd.to_datetime(df['date'], format=('%Y-%m-%d'))


In [5]:
len(train_set)

16850

In [6]:
len(test_set)

2253

In [7]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

def fit_train_scaler(train_df, 
                    outlier_validation={'5TD_return': [-0.5, 0.5]},
                    input_cols=['RSI', 'Stochastic', 'Stochastic_signal', 'ADI','OBV', 'ATR', 'ADX', 
                                    'ADX_pos', 'ADX_neg', 'MACD', 'MACD_diff', 'MACD_signal', '5TD_return', 
                                    '10TD_return', '20TD_return']):
    """
    Fits Robust Scaler on train set and returns the scaler
    INPUT: the dataframe to used to fit scaler, the outlier_validation thresholds, the columns for the scaler to be
            applied too.
    OUTPUT: fitted scaler
    """
    no_outlier_train_df = train_df.copy()
    for k, v in outlier_validation.items(): 
        no_outlier_train_df = no_outlier_train_df[no_outlier_train_df[k].between(v[0], v[1])]
    scaler = RobustScaler()
    scaler.fit(no_outlier_train_df[input_cols])
    return scaler

In [8]:
fitted_scaler = fit_train_scaler(train_set)
fitted_scaler

RobustScaler()

## Updated Build Arrays

In [17]:
a = None
if a == None:
    print('hello')

hello


In [52]:
import random

def full_dataset_randomised_arrays_NEW(unsplit_df=None,
                                       train_df=None,
                                       test_df=None,
                                       split_dataframe=True,
                                         test_set_size='3Y', 
                                         time_window=5, 
                                         stride=3, 
                                         fitted_scaler=None,
                                         check_train_outliers=False, 
                                         check_test_outliers=False, 
                                         outlier_threshold=1, 
                                         input_cols=['RSI', 'Stochastic', 'Stochastic_signal', 'ADI','OBV', 'ATR', 'ADX', 
                                                     'ADX_pos', 'ADX_neg', 'MACD', 'MACD_diff', 'MACD_signal', '5TD_return', 
                                                     '10TD_return', '20TD_return'], 
                                         target_col=['5TD_return', '10TD_return', '20TD_return'], 
                                         outlier_validation={'ATR': [-100, 100], 'Stochastic': [0, 100], 
                                                             'Stochastic_signal': [-10, 110], '5TD_return': [-0.5, 0.5]}):
    """
    A function to transform dataframe into input and output arrays.

    Takes:
    unsplit_df - unsplit dataframe that will be split into test/train
    train_df - pre-split training dataframe
    test_df - pre-split test dataframe
    split_dataframe - boolean to tell the function to split the unsplit dataframe into test/train or not. 
        Use with holdout data.
    fitted_scaler - give the function a prefitted scaler that it will use on the data.
    time_window (default=5) - time series length
    stride (default=3) - controls the number of windows taken (i.e. max_num_windows = len(df)/strides)
    check_outliers (default=False) - controls whether it checks each window for outliers or not
    input_cols (default = 'RSI', 'Stochastic', 'Stochastic_signal', 'ADI',
       'OBV', 'ATR', 'ADX', 'ADX_pos', 'ADX_neg', 'MACD', 'MACD_diff',
       'MACD_signal', '1D_past_return', '5D_past_return', '10D_past_return']) - all input features, that should 
       be included in the input array target_col (default = '5TD_return') - target variable, first 
       (newest) value for each input array
    target_col - all columns that should be included in target_col
        (default: target_col=['1D_past_return', '5D_past_return', '10D_past_return'])
    outlier_validation - a dict that sets the outlier checks to be completed. Enter data in the format:
        outlier_validation={'column_name': [lower_threshold, upper_threshold]} 
        Example: {'Stochastic': [0, 100], 'Stochastic_signal': [-10, 110], '5D_past_return': [-0.5, 0.5]}

    Return tuple (input_array, target_array).

    input_array dim: (number_of_samples x time_window x features_number)
    target_array dim: (number_of_samples x time_window x returns_numbder)
    """
    
    ## check to see if unsplit or train/test split dataframes provided
    if unsplit_df is not None:
        df = unsplit_df.copy()
        if split_dataframe == True: 
            print('Train/Test Split: Splitting unsplit dataframe')
            train_set, test_set = train_test_split_multiple_companies(unsplit_df, '3Y')
        else:
            print('Train/Test Split: Not splitting dataframe. (Holdout Data)')
            train_set= df
    elif (train_df is not None) and (test_df is not None):
        print('Train/Test Split: Using provided train/test split dataframe')
        df = train_df.copy()
        train_set = train_df.copy()
        test_set = test_df.copy()  
    else:
        print('''
        Please enter valid input dataframes. Either: 
        --> Enter a unsplit dataframe (unsplit_df=df), or 
        --> Enter pre-split train and test dataframes (train_df=train and test_df=test)
        ''')
        return None, None, None, None, None
        
    
    ## check to see if fitted scaler provided
    if fitted_scaler == None:
        print('Scaler: Fitting scaler on train set')
        scaler = fit_train_scaler(train_set, outlier_validation=outlier_validation, input_cols=input_cols)
    else:
        print('Scaler: Using provided fitted_scaler')
        scaler = fitted_scaler
    
    train_x = []
    train_y = []
    test_x = []
    test_y = []
    stats2 = []
    stats = {}
    ## go company by company
    print(f"{df['ticker'].unique().size} Companies in Dataset")
    status_count = 0
    for ticker in df['ticker'].unique():
        status_count +=1
        stats[ticker] = {}
        print(f"Starting {ticker}: Company {status_count} of {df['ticker'].unique().size}")
        train_outlier_count = 0
        test_outlier_count = 0
        company_train_x_array = []
        company_train_y_array = []
        
        company_test_x_array = []
        company_test_y_array = []

        ## train
        company_train_df = train_set[train_set['ticker'] == ticker]
        company_train_sorted = company_train_df.sort_values('date', ascending=False)
        company_train_sorted.reset_index(drop=True, inplace=True)
        for row in range(0, len(company_train_sorted), stride):
            outlier = False
            df_slice = company_train_sorted.iloc[row: row + time_window].copy()
            ## check for outliers
            if check_train_outliers == True:
                for k, v in outlier_validation.items(): 
                    if ((df_slice[k] < v[0]).any() == True) or ((df_slice[k] > v[1]).any() == True): outlier = True
                
            if df_slice.shape[0]==time_window and outlier==False:
                ## scale the window
                df_slice.loc[:, input_cols] = scaler.transform(df_slice[input_cols])
                ## add to company array
                company_train_x_array.append(np.array(df_slice[input_cols].values))
                company_train_y_array.append(np.array(df_slice[target_col].iloc[0]))
            else: train_outlier_count+=1
        
        if train_outlier_count/(len(company_train_sorted)/stride) <= outlier_threshold:
            stats[ticker]['train_possible_windows'] = (len(company_train_sorted)/stride)
            stats[ticker]['train_outliers'] = train_outlier_count
            stats[ticker]['train_windows'] = len(company_train_x_array)
            train_x.extend(company_train_x_array)
            train_y.extend(company_train_y_array)
            
        
        ## test
        if split_dataframe == True:
            company_test_df = test_set[test_set['ticker'] == ticker]
            company_test_sorted = company_test_df.sort_values('date', ascending=False)
            company_test_sorted.reset_index(drop=True, inplace=True)
            for row in range(0, len(company_test_sorted), stride):
                outlier = False
                df_slice = company_test_sorted.iloc[row: row + time_window].copy()
                ## check for outliers
                if check_test_outliers == True:
                    for k, v in outlier_validation.items(): 
                        if ((df_slice[k] < v[0]).any() == True) or ((df_slice[k] > v[1]).any() == True): outlier = True

                if df_slice.shape[0]==time_window and outlier==False:
                    ## scale the window
                    df_slice.loc[:, input_cols] = scaler.transform(df_slice[input_cols])
                    ## add to company array
                    company_test_x_array.append(np.array(df_slice[input_cols].values))
                    company_test_y_array.append(np.array(df_slice[target_col].iloc[0]))
                else: test_outlier_count+=1

            if train_outlier_count/(len(company_train_sorted)/stride) <= outlier_threshold:
                stats[ticker]['test_possible_windows'] = (len(company_test_sorted)/stride)
                stats[ticker]['test_outliers'] = test_outlier_count
                stats[ticker]['test_windows'] = len(company_test_x_array)
                test_x.extend(company_test_x_array)
                test_y.extend(company_test_y_array)
    
    print('All Companies Completed')
    print('')
    ##print('Processing Stats:', stats)
    
    ## shuffle arrays
    output_train_x = []
    output_train_y = []
    index_list = random.sample(range(len(train_x)), len(train_x))
    for i in index_list:
        output_train_x.append(train_x[i])
        output_train_y.append(train_y[i])
    if split_dataframe == True:
        return np.array(output_train_x), np.array(output_train_y), np.array(test_x), np.array(test_y), scaler
    else:
        return np.array(output_train_x), np.array(output_train_y), scaler

In [55]:
INPUT_COLS = ['RSI', 'Stochastic', 'Stochastic_signal', 'ADI','OBV', 'ATR', 'ADX', 'ADX_pos', 'ADX_neg', 'MACD', 'MACD_diff',
              'MACD_signal', '5TD_return', '10TD_return', '20TD_return']
TARGET_COLS=['5TD_return', '10TD_return', '20TD_return']
outlier_validation={'5TD_return': [-0.5, 0.5]}

stride = 100


train_x, train_y, test_x, test_y, scaler = full_dataset_randomised_arrays_NEW(unsplit_df=joined_df, 
                                                                                #train_df=train_set, 
                                                                                #test_df=test_set, 
                                                                                stride=stride,
                                                                                fitted_scaler=scaler,
                                                                                split_dataframe=True,
                                                                                input_cols=INPUT_COLS, 
                                                                                outlier_threshold=1, 
                                                                                outlier_validation=outlier_validation, 
                                                                                check_train_outliers=True,
                                                                                check_test_outliers=True, 
                                                                                target_col=TARGET_COLS, 
                                                                                time_window=6,
                                                                                test_set_size='3Y')



Train/Test Split: Splitting unsplit dataframe


c:\users\owner\.venvs\lewagon\lib\site-packages\mlchartist\preprocessing.py:125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = pd.to_datetime(df['date'], format=('%Y-%m-%d'))


Scaler: Using provided fitted_scaler
3 Companies in Dataset
Starting AAPL: Company 1 of 3
Starting GOOGL: Company 2 of 3
Starting AMZN: Company 3 of 3
All Companies Completed



In [54]:
holdout_x, holdout_y, scaler = full_dataset_randomised_arrays_NEW(unsplit_df=joined_df, 
                                                                                #train_df=train_set, 
                                                                                #test_df=test_set, 
                                                                                stride=stride,
                                                                                fitted_scaler=scaler,
                                                                                split_dataframe=False,
                                                                                input_cols=INPUT_COLS, 
                                                                                outlier_threshold=1, 
                                                                                outlier_validation=outlier_validation, 
                                                                                check_train_outliers=True,
                                                                                check_test_outliers=True, 
                                                                                target_col=TARGET_COLS, 
                                                                                time_window=6,
                                                                                test_set_size='3Y')

Train/Test Split: Not splitting dataframe
Scaler: Using provided fitted_scaler
3 Companies in Dataset
Starting AAPL: Company 1 of 3
Starting GOOGL: Company 2 of 3
Starting AMZN: Company 3 of 3
All Companies Completed



In [24]:
scaler

RobustScaler()

In [56]:
print('')
print('')
print('### Stats ###')
print('train_x', train_x.shape)
print('train_y', train_y.shape)
print('test_x', test_x.shape)
print('test_y', test_y.shape)
print('scaler', scaler)



### Stats ###
train_x (170, 6, 15)
train_y (170, 3)
test_x (24, 6, 15)
test_y (24, 3)
scaler RobustScaler()


In [58]:
print('')
print('')
print('### Stats ###')
print('holdout_x', holdout_x.shape)
print('holdout_y', holdout_y.shape)
print('scaler', scaler)



### Stats ###
holdout_x (192, 6, 15)
holdout_y (192, 3)
scaler RobustScaler()
